In [5]:
import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def scrape_data(url, selector_name, selector_price):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    names = soup.select(selector_name)
    prices = soup.select(selector_price)

    return {name.text.strip(): price.text.strip() for name, price in zip(names, prices)}

def update_sheet(data, sheet_name):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(creds)
    sheet = client.open(sheet_name).sheet1

    # Clear existing data
    sheet.clear()
    # Update headers
    sheet.update('A1', [['Cryptocurrency', 'Price']])

    # Prepare data for bulk update
    update_values = [[name, price] for name, price in data.items()]
    sheet.update('A2', update_values)

def main():
    urls = {
        'WazirX': 'https://wazirx.com/exchange/BTC-INR',
        'Mudrex': 'https://mudrex.com/coins/spotlight/top-gainers',
        'CoinSwitch': 'https://coinswitch.co/crypto-index',
        'CoinDCX': 'https://coindcx.com/markets?utm_source=coindcxwebsite&utm_medium=markets&utm_campaign=CoinDCX_HP'
    }

    # Placeholder selectors - these need to be replaced with actual selectors
    selectors = {
        'WazirX': ('.crypto-name-selector', '.crypto-price-selector'),
        'Mudrex': ('.crypto-name-selector', '.crypto-price-selector'),
        'CoinSwitch': ('.crypto-name-selector', '.crypto-price-selector'),
        'CoinDCX': ('.crypto-name-selector', '.crypto-price-selector')
    }

    all_data = {}
    for exchange, url in urls.items():
        name_selector, price_selector = selectors[exchange]
        data = scrape_data(url, name_selector, price_selector)
        all_data.update(data)

    update_sheet(all_data, 'Crypto Prices')

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'